In [ ]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 18.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import faiss
from collections import defaultdict
from scipy.spatial import distance
import time

In [ ]:
metadata = pd.read_csv('/content/test_metadata_superclass.csv')
embeddings = pd.read_csv('/content/test_embeddings_meanpool.csv')

# Merge the metadata and embeddings on 'video_id'
merged_df = pd.merge(metadata, embeddings, on='video_id')
merged_df

,video_id,start_time_seconds,end_time_seconds,labels,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,...,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127
0,iZcPr3wgJCw,30.0,40.0,['Sounds of things'],64.9,221.3,85.4,7.4,215.8,123.5,...,124.0,191.2,144.8,65.8,87.1,153.0,112.2,216.3,187.1,173.0
1,iZmn0dUXP9E,40.0,50.0,['Sounds of things'],93.7,112.2,100.1,20.7,166.8,153.3,...,54.9,181.4,106.5,192.7,50.5,67.4,95.2,123.9,190.5,132.4
2,iZ2K_GPtc6g,30.0,40.0,['Source-ambiguous sounds'],170.6,72.8,107.5,83.9,118.0,103.7,...,96.5,204.1,215.9,80.7,77.4,20.7,80.1,190.3,42.6,146.3
3,iZtF1lUPbEQ,20.0,30.0,"['Music', 'Source-ambiguous sounds', 'Animal',...",170.6,118.4,102.6,128.2,134.6,115.3,...,120.7,27.1,137.5,150.0,90.3,94.5,100.1,136.5,156.4,119.3
4,iZFRfh1Wjt8,420.0,430.0,['Sounds of things'],73.9,166.3,82.6,17.5,230.2,55.8,...,91.4,149.5,149.7,134.8,124.3,62.7,146.1,166.0,52.4,85.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21777,2Cmn2lplGfc,30.0,40.0,['Sounds of things'],59.7,189.7,13.3,18.3,179.3,130.6,...,255.0,214.7,253.7,112.6,123.7,90.4,110.9,138.4,58.5,113.1
21778,2ClQcAd-wJg,30.0,40.0,"['Sounds of things', 'Human sounds']",55.0,147.8,237.5,223.2,126.1,188.9,...,65.6,122.0,178.2,192.2,198.1,175.9,237.6,61.6,182.2,153.4
21779,-Sclnq22t7o,30.0,40.0,['Human sounds'],82.0,77.8,171.4,86.6,112.6,113.1,...,147.6,129.8,32.3,148.6,101.8,54.0,85.2,107.8,183.9,112.1
21780,-SD9DkKyOrY,30.0,40.0,['Music'],180.4,107.9,81.7,80.9,99.4,222.1,...,42.1,33.7,0.7,233.8,150.8,93.2,255.0,128.7,137.1,230.7


In [ ]:
compare_df = merged_df[['video_id', 'labels']]
compare_df['labels'] = compare_df['labels'].apply(eval)
compare_df

<ipython-input-75-4aee934afca4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compare_df['labels'] = compare_df['labels'].apply(eval)


,video_id,labels
0,iZcPr3wgJCw,[Sounds of things]
1,iZmn0dUXP9E,[Sounds of things]
2,iZ2K_GPtc6g,[Source-ambiguous sounds]
3,iZtF1lUPbEQ,"[Music, Source-ambiguous sounds, Animal, Human..."
4,iZFRfh1Wjt8,[Sounds of things]
...,...,...
21777,2Cmn2lplGfc,[Sounds of things]
21778,2ClQcAd-wJg,"[Sounds of things, Human sounds]"
21779,-Sclnq22t7o,[Human sounds]
21780,-SD9DkKyOrY,[Music]


In [ ]:
# Simplify the root_classes column for easier processing
merged_df['labels'] = merged_df['labels'].apply(lambda x: eval(x)[0] if eval(x) else None)
root_class_distribution = merged_df['labels'].value_counts()
print(root_class_distribution.head(10))

labels
Music                                  8770
Sounds of things                       5302
Human sounds                           1983
Animal                                 1949
Source-ambiguous sounds                1500
Channel, environment and background    1433
Natural sounds                          845
Name: count, dtype: int64


In [ ]:
# Select 100 audios from each of the 7 root classes
evaluation_set = pd.DataFrame()
for root_class in root_class_distribution.head(7).index:
    class_subset = merged_df[merged_df['labels'] == root_class]
    # Randomly select 100 audios if possible, or all audios if less than 100 are available
    sample_subset = class_subset.sample(n=min(100, len(class_subset)), random_state=42)
    evaluation_set = pd.concat([evaluation_set, sample_subset], axis=0)

# # Separate the dataset into evaluation set and the rest for training the models
# remaining_set = merged_df[~merged_df.index.isin(evaluation_set.index)]

In [ ]:
# Extract embeddings from dataframe
train_embeddings = merged_df.iloc[:, 4:].values
train_ids = merged_df['video_id'].tolist()
test_embeddings = evaluation_set.iloc[:, 4:].values
test_ids = evaluation_set['video_id'].tolist()
print(train_embeddings.shape)
print(test_embeddings.shape)

(21782, 128)
(700, 128)


In [ ]:
def perform_faiss_indexing(embeddings):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

In [ ]:
# perform Faiss indexing
model = perform_faiss_indexing(train_embeddings)

In [ ]:
def cal_precisionK(query, neighbors):
    query_labels = set(query)
    relevant_cnt = 0
    for neigh in neighbors:
        neighbor_labels = set(neigh)
        if not query_labels.isdisjoint(neighbor_labels):
            relevant_cnt += 1
    return relevant_cnt / len(neighbors)

In [ ]:
def faiss_evaluation(k=10):
  D, I = model.search(test_embeddings, k)
  video_ids = []
  for idxs in I:
    video_ids.append([train_ids[i] for i in idxs])
  return video_ids

In [ ]:
# collect data for faiss
query_results = faiss_evaluation()

print(len(query_results))

700


In [ ]:
precision = {}

cnt = 0
for query_id, results_ids in zip(test_ids, query_results):
  query_labels = compare_df.loc[compare_df['video_id'] == query_id]['labels'].tolist()[0]
  results_labels = []
  for results_id in results_ids:
    result_label = compare_df.loc[compare_df['video_id'] == results_id]['labels'].tolist()[0]
    results_labels.append(result_label)
  key = cnt // 100
  if key not in precision:
    precision[key] = []
  precision[key].append(cal_precisionK(query_labels, results_labels))
  cnt += 1

print(precision)

{0: [1.0, 1.0, 0.7, 0.3, 1.0, 1.0, 1.0, 0.9, 0.9, 1.0, 0.9, 0.8, 1.0, 0.5, 1.0, 0.9, 1.0, 0.9, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 0.9, 1.0, 0.8, 1.0, 1.0, 0.8, 0.7, 0.9, 1.0, 1.0, 0.9, 1.0, 0.9, 1.0, 0.5, 1.0, 0.6, 1.0, 0.7, 0.8, 1.0, 0.9, 1.0, 0.9, 1.0, 0.9, 0.9, 1.0, 1.0, 1.0, 0.9, 1.0, 0.1, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 0.8, 1.0, 0.9, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 0.9, 1.0, 0.7, 0.6, 0.9, 1.0, 1.0, 0.2, 1.0, 1.0, 1.0, 0.7, 1.0, 0.9, 1.0, 0.8, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0], 1: [0.7, 1.0, 1.0, 0.7, 0.9, 0.4, 0.9, 1.0, 0.4, 0.9, 1.0, 0.8, 0.9, 0.6, 1.0, 1.0, 0.9, 0.6, 1.0, 1.0, 0.7, 0.5, 0.6, 1.0, 0.4, 0.7, 0.4, 0.9, 0.9, 0.8, 0.9, 0.8, 0.6, 1.0, 0.8, 1.0, 0.6, 0.7, 0.9, 0.6, 0.7, 0.2, 0.6, 0.5, 1.0, 0.5, 0.3, 0.8, 0.6, 1.0, 0.9, 0.9, 0.7, 0.5, 0.9, 1.0, 0.8, 0.2, 0.9, 0.3, 1.0, 1.0, 0.2, 0.6, 1.0, 1.0, 0.5, 0.7, 1.0, 0.9, 0.9, 0.9, 1.0, 0.6, 0.9, 0.7, 0.9, 0.8, 1.0, 1.0, 0.7, 1.0, 1.0, 0.9, 1.0, 0.8, 0.9, 0.8, 1.0, 0.9, 0.9, 0.9, 0.8, 0.8, 0.5, 0.5, 0.7, 0.9, 

In [ ]:
class_precision =[np.mean(v) for v in precision.values()]
class_precision

[0.9080000000000001,
 0.7810000000000002,
 0.7609999999999999,
 0.613,
 0.604,
 0.6910000000000001,
 0.6659999999999999]

In [ ]:
print(np.mean(class_precision))

0.7177142857142859
